# Use Features

In [3]:
%cd ~/EFS

/Users/my-username/EFS


## Offline features

In [5]:
from feast import FeatureStore
store = FeatureStore(repo_path="feature_repo")
training_df = store.get_historical_features(
    entity_df=f"""
            SELECT
                driver_id,
                event_timestamp
            FROM EFS_feast_driver_hourly_stats
            WHERE event_timestamp BETWEEN (CURRENT_TIMESTAMP - INTERVAL '60' DAY) AND CURRENT_TIMESTAMP
        """,
    features=[
        "driver_hourly_stats:conv_rate",
        "driver_hourly_stats:acc_rate",
        "driver_hourly_stats:avg_daily_trips"
    ],
).to_df()
print(training_df.head())

   driver_id     event_timestamp  conv_rate  acc_rate  avg_daily_trips
0       1002 2023-01-29 07:00:00   0.401458  0.964006               97
1       1005 2023-02-05 12:00:00   0.408868  0.896338              488
2       1005 2023-01-26 02:00:00   0.934826  0.897346              566
3       1003 2023-01-28 11:00:00   0.831310  0.724465              670
4       1004 2023-01-27 01:00:00   0.077103  0.071463              197


## Online features

**To use online features you need to materialize them first:**

In [12]:
%cd ~/EFS/feature_repo

/Users/my-username/EFS/feature_repo


In [13]:
!CURRENT_TIME=$(date +'%Y-%m-%dT%H:%M:%S') && feast materialize-incremental $CURRENT_TIME

/Users/my-username/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
Materializing 2 feature views to 2023-02-09 21:00:47+00:00 into the feast_teradata.online.teradata.TeradataOnlineStore online store.

driver_hourly_stats_fresh from 2013-02-21 21:01:02+00:00 to 2023-02-09 21:00:47+00:00:
/Users/my-username/opt/anaconda3/lib/python3.9/site-packages/teradataml/context/context.py:481: TeradataMlRuntimeWarning: Warning: Password is URL encoded.
  warnings.warn("Warning: Password is URL encoded.", category=TeradataMlRuntimeWarning)
100%|█████████████████████████████████████████████████████████████████| 5/5 [00:03<00:00,  1.36it/s]
driver_hourly_stats from 2013-02-21 21:01:20+00:00 to 2023-02-09 21:00:47+00:00:
100%|█████████████████████████████████████████████████████████████████| 5/5 [00:03

**Then get acc_rate, conv_rate, avg_daily_trips for sample drivers 1001 and 1002:**

In [14]:
entity_rows = [
        {
            "driver_id": 1001,
        },
        {
            "driver_id": 1002,
        },
    ]
features_to_fetch = [
            "driver_hourly_stats:acc_rate",
            "driver_hourly_stats:conv_rate",
            "driver_hourly_stats:avg_daily_trips"
        ]
returned_features = store.get_online_features(
    features=features_to_fetch,
    entity_rows=entity_rows,
).to_dict()
for key, value in sorted(returned_features.items()):
    print(key, " : ", value)

acc_rate  :  [1.003549575805664, 0.9236539602279663]
avg_daily_trips  :  [467, 824]
conv_rate  :  [1.0, 0.7620654702186584]
driver_id  :  [1001, 1002]
